# Coronavirus Dashboard
__________________________________________
A.K.A playing with data until I'm
good enough to do something useful
(Volume 3)

By Dilan Moore
17/03/2020
__________________________________________

So this is an interesting one. Yesterday I
watched BoJo's press briefing, which was:

- All non-essential travel in London banned
- Don't go spoons for foreseeable future
- London is ahead of the curve

This begged the following questions:

- Why aren't schools closed?
- What is this curve?
- At what point on the curve do schools 
  close?
- What cool stuff can I do with the curve
- This is literally a generation defining
  statistical problem so this could be sick
  
So yeah. Let's go.

    By Dilan Moore
__________________________________________

### Step 1 - Gentle Research

So, I've started by watching lectures/videos
on the application of statistics in disease
prevention.

For the practicalities:
https://www.youtube.com/watch?v=MwDh3IEXYcU

For the maths:
https://www.youtube.com/watch?v=Kas0tIxDvrg

I'm now 

In [ ]:
# Basically, I need some coronavirus data for my